# Mixtures optimizations

## Introduction.
Mixing type optimizations allow for example to create the least expensive mixture, under cost constraint.It is also possible to maximize or minimize another parameter.
In this study, I will use several solvers, and make linear optimizations and not
linear.

Note: I worked on this type of problem for weeks, and the results and codes have been validated because they are the same as Lindo and also the problem of mixture exposed by Marc Sevaux and Christian Prins in the book "Optimization with Excel".The proposed code also strictly finds the same solution as that of the Python Pulp designer, in its famous optimization "The Whiskas Blending Problem"

Generally, percentages are used for mixing problems, which somewhat complicates the code writing with the solver.
In addition, we do not necessarily produce 100% of alloy, but this can be a unit value, for example, producing 5000 kgs of an alloy, however, this remains a mixture problem.

Making an optimization of the mixture and obtaining a range of decisions variables whose total is equal to 100% can apply to a large number of optimizations, not necessarily to liquid mixtures!

The general principle is this:
We have several suppliers for each metal, which have different prices, but their metals have
different characteristics (it can be resistance, X content, etc.).The solver will offer us the alloy at the best price.

We are not at all forced to maximize the profit, we can maximize or minimize another parameter when mixing.

Global study proposed by <b> Estelle Derrien - GitHub EstelleDerrien </b>

Creation in progress

The example of the famous linear mixture problem
"The Pittsburgh Steel Blending Problem" by Lindo,
Solved by my application www.solvgraph.com in graphic mode (yes, the solution is the same as that of the Lindo company ...):

<div style="text-align:center">
<img src="img/blend.png">
</div>

# Summary

- 1. Linear cost minimization. </b>
    - Our basic problem: Marc Sevaux and Christian Prins problem
        - Mathematical modeling
        - Solution with Python Pulp
    - Our basic problem: "The Pittsburgh Steel Problem" by Lindo
        - Mathematical modeling
        - Solution with Python Pulp
    - Our basic problem: "The Whiskas Can Blending Problem" by Python Pulp
        - Mathematical modeling
        - Solution with Python Pulp

    *************************************************************************************************
- 2. Non -linear <b> cost minimization </b>
    - Our basic problem
        - Mathematical modeling
        - Example


# Our basic problem: Marc Sevaux and Christian Prins problem

The problem exposed by Christian Prins and Marc Sevaux in the book "50 Optimizations with Excel" by Eyrolles.

In [4]:
# A company received an order of 5 tonnes of steel for boat (Christian Prins - Marc Sevaux)

# He must have the following characteristics:
# Chemical element  minimum percentage maximum percentage
# Carbon (C)        2                   3
# Copper (Cu)       0.4                 0.6
# Manganese (mn)    1.2                 1.65


# To manufacture this steel, the company has 7 raw materials from different suppliers:
 
# Raw material      C%      Cu%         mn%     available stocks (kgs) costs (euros/kilos)
# Iron alloy 1      2.5     0           1.3     4000                    1.20
# Iron alloy 2      3       0           0.8     3000                    1.50
# Iron alloy 3      0       0.3         0       6000                    0.90
# Copper alloy 1    0       90          0       5000                    1.30
# Copper alloy 2    0       96          4       2000                    1.45
# Aluminum alloy 1  0       0.4         1.2     3000                    1.20
# Aluminum alloy 2  0       0.6         0       2500                    1

# How to minimize the cost of the mixture?

# Mathematical modeling.


<div style="text-align:center">
<img src="img/blending-models.png">
</div>

# Resolution with Python Pulp
Here is my code that finds the same solution as in the book by Marc Sevaux and Christian Prins:

In [5]:
import pulp
from pulp import *

# We want to minimize the 5 tonnes of alloy #
prob = LpProblem("Production_5_tonnes",LpMinimize)

# We create our decisions
# It means that we should be what are the most judicious to choose and we give them a name for everyone
# This is the number of kg of alloy I used.
# Since the value of the number of kgs can be decimal, we specify that it is a decimal decision variable by writing continuous:
F1 = LpVariable("F1", lowBound = 0,cat='Continuous')
F2 = LpVariable("F2", lowBound = 0,cat='Continuous')
F3 = LpVariable("F3", lowBound = 0,cat='Continuous')
CU1 = LpVariable("CU1", lowBound = 0,cat='Continuous')
CU2 = LpVariable("CU2", lowBound = 0,cat='Continuous')
AL1 = LpVariable("AL1", lowBound = 0,cat='Continuous')
AL2 = LpVariable("AL2", lowBound = 0,cat='Continuous')

#Objective/min function (CI*XI) or CI is expressed in unit cost per kilo
prob += 1.20 * F1 + 1.50 * F2 + 0.90 * F3 + 1.30 * CU1 + 1.45 * CU2 +  1.20  *  AL1  +  1  *  AL2 , "Cout total"

#!We cannot use more than the stock expressed in kgs of following raw materials (see the Stock column of the 2 Second Table)
prob += F1 <= 4000; 
prob += F2 <= 3000; 
prob += F3 <= 6000; 
prob += CU1 <= 5000; 
prob += CU2 <= 2000; 
prob += AL1 <= 3000; 
prob += AL2 <= 2500; 


#!----------------- Qual requirements -------------------------------------!

#! Carbon content; 
# Here, 100 means that we need a main carbon rate of 2% of 5000 kg, it's 100 kg (see also the Doc Lindo du Fchier 7 of this REP)
# On the other hand, the percentages in the left part are kept in decimal format (idem Lindo and the example of file 3 too)!
prob += 0.025 * F1 + 0.03 * F2 >= 100; 
prob += 0.025 * F1 + 0.03 * F2 <= 150; 

##! Copper content; 
prob += 0.003 * F3 + 0.90 * CU1 + 0.96 * CU2 + 0.004 * AL1 + 0.006 * AL2  >=  20; 
prob += 0.003 * F3 + 0.90 * CU1 + 0.96 * CU2 + 0.004 * AL1 + 0.006 * AL2  <=  30; 

##!Manganese Content;Here, 60 means 1.2% of 5000 kg is equal to 60 kg, we find the same technique at Lindo
prob += 0.013 * F1 + 0.008 * F2 + 0.04 * CU2 + 0.012 * AL1   >=  60; 
prob += 0.013 * F1 + 0.008 * F2 + 0.04 * CU2 + 0.012 * AL1   <=  82.5;  


#! Finish good requirements; 
# We must specify that the total of our decisions variables expressed in KGS must be equal to 5000 kgs or 5 tonnes absolutely, it is therefore logical, no percentages here.
prob += F1 +  F2 +   F3 +  CU1 +  CU2 +   AL1 +  AL2 ==  5000 , "total"

# We display our result in the console:
prob.writeLP("monAlliage.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total cost of ingredients = ", value(prob.objective))

Status: Optimal
AL1 = 574.62426
AL2 = 0.0
CU1 = 0.0
CU2 = 27.612723
F1 = 4000.0
F2 = 0.0
F3 = 397.76302
Cout total des ingrédients =  5887.57427835


# Our basic problem: "The Pittsburgh Steel Problem" by Lindo

Source: https: //www.lindo.com/downloads/lingo_text/10-blending_of_input_materials.pdf

 Interest: We have constraints in percentages, and we now know how to manage this.
 Pulp finds the same as Lindo!So the linear program manages the percentages well.

In [6]:
import pulp
from pulp import *

# We want to minimize the cost of 2000 alloy pounds # 1 ton means 2000 pounds in England.
prob = LpProblem("Production_1_tonne",LpMinimize)

# We create our decisions
# It means that we should be what are the most judicious to choose and we give them a name for everyone
# This is the number of kg of alloy I used.
# Since the value of the number of kgs can be decimal, we specify that it is a decimal decision variable by writing continuous:
P1 = LpVariable("P1", lowBound = 0,cat='Continuous')
P2 = LpVariable("P2", lowBound = 0,cat='Continuous')
F1 = LpVariable("F1", lowBound = 0,cat='Continuous')
F2 = LpVariable("F2", lowBound = 0,cat='Continuous')
A1 = LpVariable("A1", lowBound = 0,cat='Continuous')
A2 = LpVariable("A2", lowBound = 0,cat='Continuous')
A3 = LpVariable("A3", lowBound = 0,cat='Continuous')
CB = LpVariable("CB", lowBound = 0,cat='Continuous')
S1 = LpVariable("S1", lowBound = 0,cat='Continuous')
S2 = LpVariable("S2", lowBound = 0,cat='Continuous')
S3 = LpVariable("S3", lowBound = 0,cat='Continuous')

# Objective / min function (CI * XI) or XI is expressed in cost by Pounds and CI is expressed in dollars
prob += 0.03 * P1 + 0.0645 * P2 + 0.065 * F1 + 0.061 * F2 + 0.1 * A1 +  0.13  *  A2  +  0.119  *  A3  +  0.08  *  CB  +  0.021  *  S1  +  0.02  *  S2  +  0.0195 * S3, "Cout total "

#!We cannot use more than the following stock of raw materials (see the Doc Lindo table)
prob += CB <= 20; 
prob += S1 <= 200; 
prob += S2 <= 200; 
prob += S3 <= 200; 

#!----------------- Qual requirements -------------------------------------!
#!Carbon content;Here, 60 means that we need 3% of 2000 pounds, it's 60 (see Doc Lindo's painting)
prob += 0.04 * P1 + 0.15 * CB + 0.004 * S1 + 0.001 * S2 + 0.001 * S3 >= 60; 
prob += 0.04 * P1 + 0.15 * CB + 0.004 * S1 + 0.001 * S2 + 0.001 * S3 <= 70; 
##! Chrome content; 
prob += 0.1 * P2 + 0.2 * A2 + 0.08 * A3 >=  6; 
prob += 0.1 * P2 + 0.2 * A2 + 0.08 * A3 <=  9; 
##! Manganese content; 
prob += 0.009 * P1 + 0.045 * P2 + 0.6 * A1 + 0.09 * A2 + 0.33 * A3 + 0.009 * S1 + 0.003 * S2 + 0.003 * S3 >= 27; 
prob += 0.009 * P1 + 0.045 * P2 + 0.6 * A1 + 0.09 * A2 + 0.33 * A3 + 0.009 * S1 + 0.003 * S2 + 0.003 * S3 <= 33; 
#! Silicon content; 
prob +=0.0225 * P1 + 0.15 * P2 + 0.45 * F1 + 0.42 * F2 + 0.18 * A1 + 0.3 * A2 + 0.25 * A3 + 0.3 * CB >=  54; 
prob += 0.0225 * P1 + 0.15 * P2 + 0.45 * F1 + 0.42 * F2 + 0.18 * A1 + 0.3 * A2 + 0.25 * A3 + 0.3 * CB <=  60; 


#! Finish good requirements; 
# We must specify that the total of our decisions expressed in Pounds must be equal to 2000 pounds is 1 ton absolutely, it is therefore logical, no percentages here.
prob += P1 +  P2 +  F1 +   F2 +  A1 +  A2 +   A3 +  CB + S1 + S2 + S3 ==  2000 , "total"


prob.writeLP("monAlliage.lp")
prob.solve()
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    print(v.name, "=", v.varValue)

print("Total Cost of Ingredients  = ", value(prob.objective))


Status: Optimal
A1 = 14.238863
A2 = 0.0
A3 = 0.0
CB = 0.0
F1 = 0.0
F2 = 22.062052
P1 = 1474.2641
P2 = 60.0
S1 = 200.0
S2 = 29.434959
S3 = 200.0
Total Cost of Ingredients  =  59.556293652
